<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/Pytorch003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# encoding: utf-8

import time

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 定义超参数
batch_size = 64
learning_rate = 1e-3
num_epochs = 100

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.FashionMNIST(
    root='../datasets', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.FashionMNIST(
    root='../datasets', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义 Logistic Regression 模型
class Logistic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super().__init__()
        #super(Logistic_Regression, self).__init__()
        self.logistic = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.logistic(x)
        return out


model = Logistic_Regression(28 * 28, 10)  # 图片大小是 28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 开始训练
for epoch in range(num_epochs):
    print('*' * 10)
    print(f'epoch {epoch+1}')
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    model.train()
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)  # 将图片展开成 28x28
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.item()
        _, pred = torch.max(out, 1)
        running_acc += (pred==label).float().mean()
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        with torch.no_grad():
            out = model(img)
            loss = criterion(out, label)
        eval_loss += loss.item()
        _, pred = torch.max(out, 1)
        eval_acc += (pred == label).float().mean()
    print(f'Test Loss: {eval_loss/len(test_loader):.6f}, Acc: {eval_acc/len(test_loader):.6f}')
    print(f'Time:{(time.time()-since):.1f} s')

# 保存模型
torch.save(model.state_dict(), './logstic.pth')

100%|██████████| 26421880/26421880 [00:01<00:00, 18801483.63it/s]


Extracting ../datasets/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../datasets/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 337711.44it/s]


Extracting ../datasets/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../datasets/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6153796.17it/s]


Extracting ../datasets/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../datasets/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8314315.36it/s]


Extracting ../datasets/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../datasets/FashionMNIST/raw

**********
epoch 1
[1/100] Loss: 2.074165, Acc: 0.323542
[1/100] Loss: 1.879214, Acc: 0.479245
[1/100] Loss: 1.739272, Acc: 0.539358
Finish 1 epoch, Loss: 1.724394, Acc: 0.543993
Test Loss: 1.363554, Acc: 0.659037
Time:9.1 s
**********
epoch 2
[2/100] Loss: 1.290690, Acc: 0.672031
[2/100] Loss: 1.244613, Acc: 0.674375
[2/100] Loss: 1.205874, Acc: 0.676302
Finish 2 epoch, Loss: 1.202182, Acc: 0.675873
Test Loss: 1.099027, Acc: 0.670681
Time:7.3 s
**********
epoch 3
[3/100] Loss: 1.064713, Acc: 0.685365
[3/100] Loss: 1.042297, Acc: 0.689844
[3/100] Loss: 1.022677, Acc: 0.693281
Finish 3 epoch, Loss: 1.021416, Acc: 0.693430
Test Loss: 0.977685, Acc: 0.694168
Time:9.6 s
**********
epoch 4
[4/100] Loss: 0.945484, Acc: 0.709167
[4/100] Loss: 0.935328, Acc: 0.712474
[4/100] Loss: 0.927737, Acc: 0.713420
Finish 4 epoch, Loss: 0.927337, Acc: 0.713453
Test Loss: 0.906571, Acc: 0.714769
Time:7.7 s
*